In [30]:
import pymysql 
import config as cfg
import yfinance as yf
from sqlalchemy import create_engine
import mysql.connector
import pandas as pd
import requests 
import random
import numpy as np
from datetime import datetime
import time
import warnings
warnings.filterwarnings("ignore")
import os
os.environ["TZ"] = "America/New_York" 


In [47]:
# connect to mysql db
mydb = mysql.connector.connect(
    host=cfg.db_stock["host"],
    port=cfg.db_stock["port"],
    user=cfg.db_stock["user"],
    password=cfg.db_stock["password"],
    database=cfg.db_stock["database"],
)

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}:{port}/{db}".format(
    user=cfg.db_stock["user"],
    pw=cfg.db_stock["password"],
    host=cfg.db_stock["host"],
    port=cfg.db_stock["port"],
    db=cfg.db_stock["database"]))

In [32]:
# S&P 
sp500 = pd.read_html(cfg.sp500_wiki)[0]
sp500_list=sp500["Symbol"]

## Stock Price

In [33]:
# stock price
# pull historical data for a symbol
def get_history_stock_price(symbol, max_num=1000):
    # get adjusted daily time series value
    stock = yf.Ticker(symbol)
    df = stock.history(period="max", auto_adjust=True)[-(max_num+50):].reset_index()
    
    df["Symbol"] = symbol
    df["last_close"] = df["Close"].shift(+1)
    df["Change"] = (df["Close"] - df["last_close"]) / df["last_close"]
    
    if df.empty:
        return None
    
    # macd
    df["shortEMA"] = df.Close.ewm(span=12, adjust=False).mean()
    df["longEMA"] = df.Close.ewm(span=26, adjust=False).mean()
    df["DIF"] = df["shortEMA"]-df["longEMA"]
    df["DEA"] = df["DIF"].ewm(span=9, adjust=False).mean()
    df["MACD"] = (df["DIF"]-df["DEA"])*2
     
    # RSI    
    intl = 6
    df[['upMove', 'downMove', 'avgUp', 'avgDown', 'RS', 'RSI']] = np.nan
    ## Calculate upMove & downMove
    for x in range(1, len(df)):
        df['upMove'][x] = 0
        df['downMove'][x] = 0
        if df['Close'][x] > df['Close'][x-1]:
            df['upMove'][x] = df['Close'][x] - df['Close'][x-1]
        if df['Close'][x] < df['Close'][x-1]:
            df['downMove'][x] = abs(df['Close'][x] - df['Close'][x-1])  
    ## Calculate initial avgUp & Down, RS and RSI
    df['avgUp'][intl] = df['upMove'][1:intl+1].mean()
    df['avgDown'][intl] = df['downMove'][1:intl+1].mean()
    df['RS'][intl] = df['avgUp'][intl] / df['avgDown'][intl]
    df['RSI'][intl] = 100 - (100/(1+df['RS'][intl]))
    ## Calculate rest of avgUp, avgDown, RS, RSI
    for x in range(intl+1, len(df)):
        df['avgUp'][x] = (df['avgUp'][x-1]*(intl-1)+df['upMove'][x])/intl
        df['avgDown'][x] = (df['avgDown'][x-1]*(intl-1)+df['downMove'][x])/intl
        df['RS'][x] = df['avgUp'][x] / df['avgDown'][x]
        df['RSI'][x] = 100 - (100/(1+df['RS'][x]))
        
    # combine all
    df = df[["Date", "Symbol", "Open", "High", "Low", "Close", "Volume", "Change", "DIF", "DEA", "MACD", "RSI"]][-max_num:].dropna().reset_index(drop=True)
    
    return(df)

def get_sp500_history_stock_price(sp500_list, max_num=1000):
    df = pd.DataFrame()
    for symbol in sp500_list:
        print(symbol)
        df_symbol = get_history_stock_price(symbol, max_num=max_num)
        df = df.append([df_symbol])
    df = df[["Date", "Symbol", "Open", "High", "Low", "Close", "Volume", "Change", "DIF", "DEA", "MACD", "RSI"]]
    df.dropna(subset=["Symbol", "Close"], inplace=True)
    return(df)


In [34]:
df_stock_price = get_sp500_history_stock_price(sp500_list)

MMM
ABT
ABBV
ABMD
ACN
ATVI
ADBE
AMD
AAP
AES
AFL
A
APD
AKAM
ALK
ALB
ARE
ALGN
ALLE
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
ABC
AME
AMGN
APH
ADI
ANSS
ANTM
AON
AOS
APA
AAPL
AMAT
APTV
ADM
ANET
AJG
AIZ
T
ATO
ADSK
ADP
AZO
AVB
AVY
BKR
BLL
BAC
BBWI
BAX
BDX
BRK.B
- BRK.B: No data found, symbol may be delisted
BBY
BIO
TECH
BIIB
BLK
BK
BA
BKNG
BWA
BXP
BSX
BMY
AVGO
BR
BRO
BF.B
- BF.B: No data found for this date range, symbol may be delisted
CHRW
CDNS
CZR
CPB
COF
CAH
KMX
CCL
CARR
CTLT
CAT
CBOE
CBRE
CDW
CE
CNC
CNP
CDAY
CERN
CF
CRL
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CTXS
CLX
CME
CMS
KO
CTSH
CL
CMCSA
CMA
CAG
COP
ED
STZ
COO
CPRT
GLW
CTVA
COST
CTRA
CCI
CSX
CMI
CVS
DHI
DHR
DRI
DVA
DE
DAL
XRAY
DVN
DXCM
FANG
DLR
DFS
DISCA
DISCK
DISH
DG
DLTR
D
DPZ
DOV
DOW
DTE
DUK
DRE
DD
DXC
EMN
ETN
EBAY
ECL
EIX
EW
EA
EMR
ENPH
ETR
EOG
EFX
EQIX
EQR
ESS
EL
ETSY
EVRG
ES
RE
EXC
EXPE
EXPD
EXR
XOM
FFIV
FAST
FRT
FDX
FIS
FITB
FE
FRC
FISV
FLT
FMC
F
FTNT
FTV
FBHS
FOXA
FOX
BEN
FCX
GPS
GRMN
IT
GNRC


In [35]:
print(df_stock_price.shape)
df_stock_price.to_csv("yfinance_price.csv", index=False)

(499197, 12)


In [36]:
df_stock_price.to_sql("Price", con=engine, index=False, if_exists="replace")
engine.execute("SHOW TABLES").fetchall()


[('Price',), ('abcd',), ('test',)]

In [37]:
mycursor = mydb.cursor()
mycursor.execute(
    """
    SELECT *
    FROM Price
    LIMIT 5
    """
)
print([i[0] for i in mycursor.description])
result = mycursor.fetchall()
for x in result:
    print(x)

['Date', 'Symbol', 'Open', 'High', 'Low', 'Close', 'Volume', 'Change', 'DIF', 'DEA', 'MACD', 'RSI']
(datetime.datetime(2017, 11, 24, 0, 0), 'MMM', 205.993438589389, 206.00231159219456, 204.91033619806876, 205.4163818359375, 659100.0, -0.0008635275430403725, 2.144359748716937, 2.304850072856183, -0.3209806482784918, 62.33685630737739)
(datetime.datetime(2017, 11, 27, 0, 0), 'MMM', 205.7448629232553, 208.21291240663504, 205.19443871441462, 207.7423858642578, 1780800.0, 0.011323361883464833, 2.295000693506722, 2.302880196986291, -0.015759006959137345, 73.14076380699487)
(datetime.datetime(2017, 11, 28, 0, 0), 'MMM', 207.84886965546596, 209.46464990915248, 207.1919153714291, 209.18943786621094, 1871700.0, 0.0069656078894687, 2.502304719426519, 2.3427651014743365, 0.3190792359043648, 77.8781667574436)
(datetime.datetime(2017, 11, 29, 0, 0), 'MMM', 209.56235707173073, 211.46222282232958, 209.19836074583006, 211.3024139404297, 1754600.0, 0.01010077801141243, 2.804762606401937, 2.4351646024598

## Fundamentals

In [20]:
def get_stock_fundamentals(symbol):
    stock = yf.Ticker(symbol)
    return(stock.get_info())

def get_sp500_funamentals(sp500_list):
    info = []
    for symbol in sp500_list:
        print(symbol)
        data = get_stock_fundamentals(symbol)
        if "symbol" in data:
            info.append(data)
    
    df = pd.DataFrame(info)
    print("done!")
    return(df)

In [21]:
df_fundamentals = get_sp500_funamentals(sp500_list=sp500_list)


MMM
ABT
ABBV
ABMD
ACN
ATVI
ADBE
AMD
AAP
AES
AFL
A
APD
AKAM
ALK
ALB
ARE
ALGN
ALLE
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
ABC
AME
AMGN
APH
ADI
ANSS
ANTM
AON
AOS
APA
AAPL
AMAT
APTV
ADM
ANET
AJG
AIZ
T
ATO
ADSK
ADP
AZO
AVB
AVY
BKR
BLL
BAC
BBWI
BAX
BDX
BRK.B
BBY
BIO
TECH
BIIB
BLK
BK
BA
BKNG
BWA
BXP
BSX
BMY
AVGO
BR
BRO
BF.B
CHRW
CDNS
CZR
CPB
COF
CAH
KMX
CCL
CARR
CTLT
CAT
CBOE
CBRE
CDW
CE
CNC
CNP
CDAY
CERN
CF
CRL
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CTXS
CLX
CME
CMS
KO
CTSH
CL
CMCSA
CMA
CAG
COP
ED
STZ
COO
CPRT
GLW
CTVA
COST
CTRA
CCI
CSX
CMI
CVS
DHI
DHR
DRI
DVA
DE
DAL
XRAY
DVN
DXCM
FANG
DLR
DFS
DISCA
DISCK
DISH
DG
DLTR
D
DPZ
DOV
DOW
DTE
DUK
DRE
DD
DXC
EMN
ETN
EBAY
ECL
EIX
EW
EA
EMR
ENPH
ETR
EOG
EFX
EQIX
EQR
ESS
EL
ETSY
EVRG
ES
RE
EXC
EXPE
EXPD
EXR
XOM
FFIV
FAST
FRT
FDX
FIS
FITB
FE
FRC
FISV
FLT
FMC
F
FTNT
FTV
FBHS
FOXA
FOX
BEN
FCX
GPS
GRMN
IT
GNRC
GD
GE
GIS
GM
GPC
GILD
GL
GPN
GS
GWW
HAL
HBI
HIG
HAS
HCA
PEAK
HSIC
HSY
HES
HPE
HLT
HOLX
HD
HON
HRL
HST
HWM
HPQ
HUM

In [42]:
kp_list = [
    "symbol",
    "quoteType",
    "shortName",
    "longName",
    "exchange",
    "exchangeTimezoneName",
    "currency",
    "sector",
    "industry",
    "fullTimeEmployees",
    "longBusinessSummary", 
    "country", 
    "state", 
    "city", 
    "phone", 
    "website", 
    "address1", 
    "fiftyTwoWeekLow",   
    "fiftyTwoWeekHigh", 
    "pegRatio", 
    "marketCap", 
    "enterpriseValue",
    "bookValue",
    "averageVolume", 
    "floatShares", 
    "sharesOutstanding", 
    "enterpriseToRevenue", 
    "beta", 
    "trailingPE", 
    "trailingEps", 
    "priceToSalesTrailing12Months", 
    "forwardEps", 
    "forwardPE",
    "priceToBook",
    "heldPercentInstitutions", 
    "debtToEquity", 
    "returnOnEquity", 
    "returnOnAssets",
    "shortRatio", 
    "sharesShort", 
    "52WeekChange", 
    "enterpriseToEbitda", 
    "totalCash", 
    "totalDebt", 
    "totalRevenue",   
    "totalCashPerShare", 
    "numberOfAnalystOpinions", 
    "currentPrice", 
    "targetHighPrice", 
    "targetMeanPrice", 
    "targetMedianPrice", 
    "targetLowPrice", 
    "recommendationKey", 
    "recommendationMean", 
    "revenuePerShare", 
    "grossProfits", 
    "freeCashflow", 
    "ebitda", 
    "operatingMargins", 
    "revenueGrowth", 
    "operatingCashflow",  
    "grossMargins", 
    "profitMargins", 
    "ebitdaMargins",  
]

In [36]:
df_fundamentals.dropna(subset=["currentPrice"], inplace=True)
df_fundamentals = datetime.now()
df_fundamentals.to_csv("sp500_{}_{}_{}.csv".format(dt.year, dt.month, dt.day), index=False)

In [44]:
df_fundamentals.to_sql("Fundamentals", con=engine, index=False, if_exists="replace")
engine.execute("SHOW TABLES").fetchall()


[('Fundamentals',), ('Price',), ('abcd',), ('test',)]

In [50]:
mycursor = mydb.cursor()
mycursor.execute(
    """
    SELECT sector, count(*) as cnt
    FROM Fundamentals
    GROUP BY sector;
    """
)
print([i[0] for i in mycursor.description])
result = mycursor.fetchall()
for x in result:
    print(x)

['sector', 'cnt']
('Industrials', 73)
('Healthcare', 65)
('Technology', 71)
('Communication Services', 27)
('Consumer Cyclical', 66)
('Utilities', 28)
('Financial Services', 68)
('Basic Materials', 21)
('Real Estate', 29)
('Consumer Defensive', 34)
('Energy', 21)
